In [162]:
import numpy as np
import trimesh
from pathlib import Path
import csv
from collections import Counter
import time
from timeit import timeit

In [56]:
file_name = "neuron-775959265587_part_2"
mesh_file_location = "/notebooks/18_Spine_Allen/neurons"
labels_file_location = "/notebooks/18_Spine_Allen/cgal"

clusters = "12"
smoothness = "0.04"

labels_file = str(Path(labels_file_location) / Path(file_name + "-cgal_" + str(clusters) + "_" + str(smoothness) + ".csv" ))  
sdf_file = str(Path(labels_file_location) / Path(file_name + "-cgal_" + str(clusters) + "_" + str(smoothness) + "_sdf.csv" ))  


In [31]:
#using insert function instead of append
labels_list = []
with open(labels_file) as csvfile:
    print("inside labels file")

    for i,row in enumerate(csv.reader(csvfile)):
        labels_list.append(int(row[0]))

inside labels file


In [32]:
myCounter = Counter(labels_list)
key_list = list(myCounter.keys())
key_list_np = np.array(key_list)
np.max(key_list_np)

1814

In [61]:
sdf_labels = np.zeros(len(labels_list)).astype("float")
segmentation_length = len(list(myCounter.keys()))
with open(sdf_file) as csvfile:

    for i,row in enumerate(csv.reader(csvfile)):
        sdf_labels[i] = float(row[0])


sdf_temp_dict = {}
for i in range(0,segmentation_length):
    sdf_temp_dict[i] = []

#print("sdf_temp_dict = " + str(sdf_temp_dict))
#print("sdf_labels = " + str(sdf_labels))
#iterate through the labels_list
for i,label in enumerate(labels_list):
    sdf_temp_dict[label].append(sdf_labels[i])
#print(sdf_temp_dict)

#now calculate the stats on the sdf values for each label
sdf_final_dict = {}

for dict_key,value in sdf_temp_dict.items():

    #just want to store the median
    sdf_final_dict[dict_key] = np.median(value)



In [27]:
unique_key_list_np = np.unique(key_list_np)
unique_key_list_np.size

1815

In [43]:
my_array = np.asarray([list(myCounter.keys()),list(myCounter.values())])

In [83]:
max_backbone_threshold  = 200 
backbone_threshold = 40
backbone_width_threshold = 0.1

In [53]:
start_time = time.time()
backbone_labels_np = np.array(list(myCounter.keys()))[np.where(np.array(list(myCounter.values())) > max_backbone_threshold)]
print(f"Total time = {time.time() - start_time}")


spine_labels = []
backbone_labels = []

#put groups that have number of faces > max_backbone_threshold into backbone list
start_time = time.time()
for label,times in myCounter.items():
    if(times >= max_backbone_threshold):
        #print(str(label) + ":" + str(times))
        backbone_labels.append(label)   
        
print(f"Total time = {time.time() - start_time}")






Total time = 0.001603841781616211
Total time = 0.0007765293121337891


In [110]:

start_time = time.time()
total_items = np.array(sorted(myCounter.items()))
keys = total_items[:,0]
values = total_items[:,1]
big_threshold = values >= max_backbone_threshold

small_threshold = values > backbone_threshold 
sdf_threshold = np.array(list(sdf_final_dict.values())) >= backbone_width_threshold
total_list = np.logical_or(big_threshold,np.logical_and(small_threshold,sdf_threshold))
backbone_labels_new = keys[total_list]

print(f"Total time = {time.time() - start_time}")

#print(f"Total time = {time.time() - start_time}")


Total time = 0.006491184234619141


In [111]:
spine_labels = []
backbone_labels = []

#faster this way than with numpy way

#put groups that have number of faces > max_backbone_threshold into backbone list
start_time = time.time()
for label,times in myCounter.items():
    if(times >= max_backbone_threshold):
        #print(str(label) + ":" + str(times))
        backbone_labels.append(label)   

#put groups that have median sdf value > backbone_width_threshold AND greater than backbone_threshold into backbone list
for label in myCounter.keys():
    if( sdf_final_dict[label] >= backbone_width_threshold):
        #print(str(label) + ":" + str(times))
        if(myCounter[label] > backbone_threshold) and (label not in backbone_labels):
            backbone_labels.append(label) 

print(f"Initial backbone list append took: {time.time() - start_time} seconds")


Initial backbone list append took: 0.018411874771118164 seconds


In [112]:
print(len(backbone_labels_new),len(np.array(backbone_labels)))
np.array_equal(backbone_labels_new,np.array(backbone_labels))
counter = []
for x in backbone_labels:
    if not np.isin(x,backbone_labels_new):
        counter.append(x)
counter

873 873


[]

In [104]:
myCounter[16]

200

In [102]:
#total_list = np.logical_or(big_threshold,np.logical_and(small_threshold,sdf_threshold))
sdf_threshold[16]

False

In [91]:
myCounter[1814]

10387

In [76]:
#keys  = np.array(sorted(myCounter.items()))[:,0]


In [69]:
np.array(list(myCounter.values())) > backbone_threshold np.array(list(sdf_final_dict.values())) > width_threshold

array([False, False, False, ...,  True,  True,  True])

AttributeError: 'dict_values' object has no attribute 'tolist'

In [77]:
np.array(sdf_final_dict.values()) > width_threshold

TypeError: '>' not supported between instances of 'dict_values' and 'float'

# OPTOMIZING FIND NEIGHBORS

In [160]:
full_path = str(Path(mesh_file_location) / Path(file_name))
mesh = trimesh.load_mesh(full_path + ".off")
current_label = 17

In [174]:
labels_list = np.array(labels_list)

In [181]:
length = len(mesh.faces)

#indexes = np.array([x for x in range(0,len(mesh.faces))])
indexes = np.linspace(0,length,length + 1).astype(int)
np.arange(length)

array([     0,      1,      2, ..., 238331, 238332, 238333])

In [184]:
start_time = time.time()
row1 = mesh.face_adjacency[:,0]
row2 = mesh.face_adjacency[:,1]

#indexes = np.array([x for x in range(0,len(mesh.faces))])
#indexes = np.linspace(0,length,length + 1)
indexes = np.arange(len(mesh.faces))
print(f"Total time: {(time.time() - start_time)*2000} seconds")

indexes_mask = labels_list == current_label
print(f"Total time: {(time.time() - start_time)*2000} seconds")


Total time: 2.17437744140625 seconds
Total time: 4.546165466308594 seconds


In [185]:
start_time = time.time()
faces_indeces = indexes[indexes_mask]
print(f"Total time: {(time.time() - start_time)*2000} seconds")
row_1_members = np.isin(row1,faces_indeces)
row_2_members = np.isin(row2,faces_indeces)
print(f"Total time: {(time.time() - start_time)*2000} seconds")

Total time: 1.0957717895507812 seconds
Total time: 312.61253356933594 seconds


In [ ]:
#one adjacency search


In [143]:
print(sum(row_1_members))
print(sum(row_2_members))

255
255


In [186]:
start_time = time.time()
mask = np.logical_xor(row_1_members,row_2_members)
neighbor_faces = mesh.face_adjacency[mask,:]
neighbor_faces_labels = labels_list_array[neighbor_faces.ravel()]
neighbors_dict = dict(Counter(neighbor_faces_labels[neighbor_faces_labels != 17]))
print(f"Total time: {(time.time() - start_time)*2000} seconds")
neighbors = list(neighbors_dict.keys())
print(f"Total time: {(time.time() - start_time)*2000} seconds")
neighbors

Total time: 7.380962371826172 seconds
Total time: 7.992267608642578 seconds


[255]

In [149]:
labels_list_array[mesh.face_adjacency[mask,:].ravel()]

array([255,  17, 255,  17,  17, 255,  17, 255,  17, 255, 255,  17])

In [168]:
start_time = time.time()
Counter(labels_list)[current_label]
print(f"Total time: {(time.time() - start_time)*2000} seconds")

Total time: 83.29391479492188 seconds


In [194]:
x = np.array([[1,3],[4,5]])
a,b = x.T

In [213]:
backbone_labels = list(labels_list)
#print(type(backbone_labels))

to_remove = list(labels_list)[:100]
#print(to_remove)

start_time = time.time()
final_backbones = [bkbone for bkbone in backbone_labels if bkbone not in to_remove]
print(f"Total time: {(time.time() - start_time)*2000} seconds")

start_time = time.time()
final_backbones_set = list(set(backbone_labels).difference(set(to_remove)))
print(f"Total time: {(time.time() - start_time)*2000} seconds")

print(len(final_backbones),len(final_backbones_set))

Total time: 1117.1150207519531 seconds
Total time: 24.948596954345703 seconds
234909 1810


In [205]:
labels_list

array([1777, 1777, 1777, ..., 1525, 1525, 1525])

In [214]:
x = set()

In [227]:
x.add(1)
x.add(3)
x.add(5)

In [228]:
len(x)

3

In [223]:
1 in x

True

In [226]:
x.pop()

1

In [229]:
x.append(set(6,7,9))

AttributeError: 'set' object has no attribute 'append'

In [242]:
y = {6,7,9}

In [243]:
x.update(set({6,7,9}))

In [244]:
x

{1, 3, 5, 6, 7, 9}

In [245]:
x.difference(y)

{1, 3, 5}

In [236]:
type(set())

set

In [241]:
for x in None:
    print("hello")

TypeError: 'NoneType' object is not iterable

In [239]:
if None not in set():
    print("hello")

hello


In [246]:
new_backbone_labels = [0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 26, 27, 28, 29, 30, 31, 32, 33, 36, 38, 39, 40, 41, 42, 43, 45, 48, 51, 52, 53, 55, 68, 70, 84, 85, 86, 87, 89, 90, 91, 94, 95, 96, 98, 99, 100, 104, 107, 108, 109, 110, 113, 122, 128, 131, 133, 134, 135, 138, 139, 140, 142, 144, 145, 147, 155, 159, 168, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 192, 193, 195, 196, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 216, 217, 218, 219, 220, 223, 224, 225, 226, 228, 229, 231, 235, 265, 268, 270, 272, 273, 277, 278, 279, 280, 281, 283, 284, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 298, 299, 300, 301, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 316, 317, 318, 319, 322, 323, 324, 327, 328, 330, 332, 336, 339, 340, 343, 344, 346, 347, 348, 349, 350, 351, 352, 353, 355, 356, 357, 358, 359, 360, 361, 363, 367, 368, 369, 371, 374, 376, 377, 379, 380, 382, 383, 384, 389, 396, 401, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 419, 420, 423, 425, 427, 428, 429, 431, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 445, 448, 449, 450, 454, 456, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 481, 483, 484, 485, 486, 489, 490, 493, 495, 496, 499, 500, 501, 502, 505, 507, 508, 509, 513, 518, 522, 523, 525, 529, 533, 539, 548, 565, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 579, 580, 581, 582, 585, 586, 588, 589, 591, 592, 593, 598, 599, 600, 601, 602, 604, 605, 606, 608, 609, 611, 613, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 634, 635, 636, 637, 638, 639, 640, 641, 642, 644, 645, 650, 651, 658, 660, 661, 663, 668, 669, 670, 671, 672, 675, 677, 678, 679, 680, 681, 682, 683, 684, 686, 688, 690, 691, 693, 696, 699, 700, 702, 703, 704, 705, 707, 708, 709, 710, 711, 712, 713, 715, 716, 717, 718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 735, 737, 738, 741, 742, 744, 747, 748, 760, 761, 763, 764, 768, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 785, 791, 792, 793, 794, 795, 796, 798, 800, 801, 803, 804, 806, 810, 815, 819, 824, 825, 827, 828, 829, 830, 832, 833, 834, 835, 836, 837, 839, 841, 844, 847, 850, 852, 854, 857, 858, 859, 860, 863, 872, 874, 875, 882, 902, 903, 904, 905, 906, 907, 908, 909, 911, 912, 914, 915, 916, 917, 918, 920, 922, 923, 926, 927, 928, 929, 931, 932, 933, 936, 941, 946, 948, 949, 951, 953, 954, 955, 957, 958, 959, 962, 963, 965, 967, 969, 970, 975, 978, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1014, 1017, 1020, 1024, 1025, 1026, 1027, 1031, 1032, 1034, 1040, 1043, 1044, 1053, 1061, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1104, 1105, 1106, 1107, 1113, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1134, 1135, 1136, 1138, 1139, 1140, 1142, 1143, 1145, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1157, 1158, 1160, 1162, 1163, 1164, 1165, 1167, 1171, 1172, 1173, 1174, 1177, 1181, 1182, 1184, 1186, 1187, 1188, 1190, 1191, 1192, 1193, 1195, 1196, 1197, 1198, 1199, 1200, 1202, 1205, 1206, 1208, 1209, 1212, 1214, 1215, 1218, 1226, 1227, 1229, 1237, 1239, 1242, 1243, 1245, 1256, 1264, 1273, 1276, 1285, 1288, 1289, 1290, 1292, 1294, 1295, 1297, 1298, 1299, 1301, 1302, 1307, 1312, 1314, 1319, 1331, 1333, 1334, 1335, 1336, 1337, 1339, 1340, 1341, 1342, 1343, 1345, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1358, 1363, 1364, 1367, 1368, 1370, 1371, 1374, 1376, 1378, 1380, 1385, 1390, 1391, 1393, 1398, 1405, 1413, 1417, 1421, 1429, 1431, 1432, 1435, 1436, 1438, 1439, 1440, 1441, 1442, 1445, 1450, 1458, 1459, 1462, 1463, 1464, 1465, 1467, 1471, 1481, 1482, 1486, 1487, 1491, 1492, 1500, 1503, 1506, 1507, 1530, 1531, 1532, 1534, 1535, 1539, 1541, 1542, 1545, 1546, 1548, 1551, 1554, 1560, 1567, 1569, 1570, 1572, 1575, 1576, 1577, 1578, 1579, 1581, 1582, 1585, 1586, 1589, 1591, 1605, 1606, 1619, 1620, 1621, 1626, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1641, 1644, 1645, 1646, 1647, 1648, 1649, 1651, 1653, 1655, 1671, 1673, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1699, 1700, 1701, 1702, 1705, 1706, 1709, 1711, 1713, 1714, 1715, 1723, 1724, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1741, 1743, 1744, 1745, 1748, 1749, 1753, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1766, 1767, 1768, 1770, 1771, 1772, 1778, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1788, 1789, 1790, 1791, 1793, 1795, 1799, 1800, 1801, 1802, 1806]

In [253]:
a = [1,4,5,6]
a + [5,7,9]

[1, 4, 5, 6, 5, 7, 9]

In [254]:
a

[1, 4, 5, 6]